In [1]:
# loading common tools, plotting, bson, progress bar, etc
import os, sys, math, io
import numpy as np
import pandas as pd
import multiprocessing as mp
import bson
import struct
from PIL import Image
import time
import shutil

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict
from tqdm import *

In [2]:
# loading PyTorch
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torchvision
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from torch.utils.data import Dataset

In [3]:
train_offsets_df = pd.read_csv("train_offsets.csv", index_col=0)
train_images_df = pd.read_csv("train_freqs.csv")
val_images_df = pd.read_csv("val_images_all.csv", index_col=0)
categories_df = pd.read_csv('categories_name_to_id.csv')

/home/hua/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
train_images_df.head()

,product_id,category_idx,img_idx,freqs
0,0,5055,0,61688
1,1,5055,0,61688
2,5,5055,0,61688
3,11,5055,0,61688
4,16,5055,0,61688


In [5]:
train_images_df.tail()

,product_id,category_idx,img_idx,freqs
12129136,19028368,5101,2,13
12129137,20643558,5101,0,13
12129138,20643558,5101,1,13
12129139,20643558,5101,2,13
12129140,20643558,5101,3,13


In [6]:
idx2l1 = list(categories_df['category_level1'])
idx2l2 = list(categories_df['category_level2'])
frequencies = train_offsets_df['category_id'].value_counts()

In [7]:
data_dir = "/media/hua/HuaSSD/KaggleData/CdiscountImageClassificationChallenge/"
train_bson_path = os.path.join(data_dir, "train.bson")
train_bson_file = open(train_bson_path, "rb")

In [8]:
class BSONIterator(Dataset):
    def __init__(self, bson_file, images_df, offsets_df, transform, mode = 'train'):
        super(BSONIterator, self).__init__()
        self.file = bson_file
        self.images_df = images_df
        self.offsets_df = offsets_df
        self.transform = transform
        self.mode = mode

    def __getitem__(self, idx):
        image_row = self.images_df.iloc[idx]
        product_id = image_row["product_id"]
        offset_row = self.offsets_df.loc[product_id]
        # Random access this product's data from the BSON file.
        self.file.seek(offset_row["offset"])
        item_data = self.file.read(offset_row["length"])
        # Grab the image from the product.
        item = bson.BSON.decode(item_data)
        img_idx = image_row["img_idx"]
        bson_img = item["imgs"][img_idx]["picture"]

        # Load the image.
        image = io.BytesIO(bson_img)
        img = Image.open(image)
        x = self.transform(img)
        idx = int(image_row["category_idx"])
        level1 = int(idx2l1[idx])
        level2 = int(idx2l2[idx])

        target1 = torch.LongTensor([level1])
        target2 = torch.LongTensor([level2])
        target3 = torch.LongTensor([idx])
        if self.mode == 'train':
            return x, target1, target2, target3 #for the sake of pin_memory and async
        if self.mode == 'valid':
            return x, [target1]*10, [target2]*10, [target3]*10
        else:
            return x
    
    def __len__(self):
        return len(self.images_df)

In [9]:
from tv_transforms import *

In [10]:
mean, std = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
transform_train = T.Compose([T.RandomHorizontalFlip(), 
                             T.ToTensor(),T.Normalize(mean=mean, std=std)])
transform_val = T.Compose([T.ToTensor(),T.Normalize(mean=mean, std=std)])

Create a generator for training and a generator for validation.

In [11]:
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df, transform_train, mode = 'train')
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df, transform_val, mode = 'train')

In [12]:
print(len(train_gen), len(val_gen))

12129141 242152


In [13]:
# cutoff = 1000
# freqs = train_images_df['freqs']
# weights = np.where(freqs>cutoff, 1.0, 2.0)
# num_samples = len(train_gen)

In [14]:
batch_size = 128
loader_train = DataLoader(train_gen, batch_size=batch_size, 
                          sampler=sampler.RandomSampler(train_gen), 
                          num_workers=1, pin_memory = True)
loader_val = DataLoader(val_gen, batch_size=batch_size, sampler=sampler.SequentialSampler(val_gen), 
                        num_workers=1, pin_memory = True)

In [15]:
print(len(loader_train), len(loader_val))

94759 1892


In [16]:
# itr = iter(loader_train)

In [17]:
# img, target1, target2, target3 = next(itr)

In [18]:
# img.size(), target1.size(), target2.size(), target3.size()

In [19]:
# itr = iter(loader_val)

In [20]:
# img, target1, target2, target3 = next(itr)

In [21]:
# img.size(), target1.size(), target2.size(), target3.size()

# Training

**ResNet101**

In [22]:
model = torchvision.models.resnet101(pretrained=True)
#0.5 comparable to the tencrop method with 4X4/6X6
model.avgpool = nn.Sequential(nn.AvgPool2d(kernel_size = 6), nn.Dropout(p=0.5,inplace=True))
model.fc = nn.Linear(in_features=2048, out_features=49 + 483 + 5270)

In [23]:
trained_model = 'model_best.pth.tar'

def load_model(model, trained_model):
    if os.path.isfile(trained_model):
        print("=> loading checkpoint '{}'".format(trained_model))
        checkpoint = torch.load(trained_model)
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint '{}'".format(trained_model))
        return model
    else:
        print("=> no checkpoint found at '{}'".format(best_model))

In [24]:
model = load_model(model, trained_model)

=> loading checkpoint 'model_best.pth.tar'
=> loaded checkpoint 'model_best.pth.tar'


In [25]:
for layer in [model.conv1, model.bn1, model.relu, model.maxpool, model.layer1, model.layer2, model.layer3]:
    for param in layer.parameters():
        param.requires_grad = False
model.cuda()

ResNet (
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (relu): ReLU (inplace)
  (maxpool): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
  (layer1): Sequential (
    (0): Bottleneck (
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (relu): ReLU (inplace)
      (downsample): Sequential (
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (1): Bott

In [26]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(lr, optimizer, epoch, denominator = 2):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // denominator))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    batch_size = target.size(0)
    _, pred = output.max(dim=1)
    correct = pred.eq(target)
    res = []
    for k in topk:
        correct_k = correct.float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [27]:
def train(train_loader, model, criterion, optimizer, weights, epoch, print_freq = 50):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    loss_log = []
    acc_log = []
    PREC1 = 1

    # switch to train mode
    model.train()

    end = time.time()
    for i, (img, target1, target2, target3) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        
        img = img.cuda(async=True)
        img_var = Variable(img)
        
        target1 = target1.view(-1).cuda(async=True)
        target1_var = Variable(target1)
        target2 = target2.view(-1).cuda(async=True)
        target2_var = Variable(target2)
        target3 = target3.view(-1).cuda(async=True)
        target3_var = Variable(target3)

        # compute output
        output = model(img_var)
        loss1 = criterion(output[:, :49], target1_var)
        loss2=  criterion(output[:, 49:532], target2_var)
        loss3=  criterion(output[:, 532:], target3_var)
        loss = loss1*weights[0] + loss2*weights[1] + loss3*weights[2]
        # measure accuracy and record loss of selected target
        if weights[2]>0:
            prec1 = accuracy(output.data[:, 532:], target3, topk=(1, ))[0]#only need top1
        elif weights[1]>0:
            prec1 = accuracy(output.data[:, 49:532], target2, topk=(1, ))[0]
        else:
            prec1 = accuracy(output.data[:, :49], target1, topk=(1, ))[0]
        losses.update(loss.data[0], img.size(0)) #[0] to take out the float inside torch.Tensor
        top1.update(prec1[0], img.size(0))
        loss_log.append(losses.val)
        acc_log.append(top1.val)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))
    return loss_log, acc_log

In [28]:
def validate(val_loader, model, weights, print_freq=20):
    batch_time = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (img, target1, target2, target3) in enumerate(val_loader):
        
        img = img.cuda(async=True)
        img_var = Variable(img, volatile=True)
        
        target1 = target1.view(-1).cuda(async=True)
        target1_var = Variable(target1)
        target2 = target2.view(-1).cuda(async=True)
        target2_var = Variable(target2)
        target3 = target3.view(-1).cuda(async=True)
        target3_var = Variable(target3)

        # compute output
        output = model(img_var)

        # measure accuracy and record loss
        if weights[2]> 0:
            prec1 = accuracy(output.data[:, 532:], target3, topk=(1, ))[0]#only need top1
        elif weights[1]>0:
            prec1 = accuracy(output.data[:, 49:532], target2, topk=(1, ))[0]
        else:
            prec1 = accuracy(output.data[:, :49], target1, topk=(1, ))[0]
        top1.update(prec1[0], img.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg

In [29]:
def validate_average(val_loader, model, weights, n = 10, print_freq=80):
    batch_time = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (img, target1, target2, target3) in enumerate(val_loader):
        
        img = img.cuda(async=True)
        img_var = Variable(img, volatile=True)
        
        target1 = target1.view(-1, n)[:, 0]
        target1 = target1.cuda()
        target1_var = Variable(target1)
        target2 = target2.view(-1, n)[:, 0]
        target2 = target2.cuda()
        target2_var = Variable(target2)
        target3 = target3.view(-1, n)[:, 0]
        target3 = target3.cuda()
        target3_var = Variable(target3)

        # compute output
        output = model(img_var)
        output = output.view(-1, n, 5802).mean(dim = 1)

        # measure accuracy and record loss
        if weights[2]> 0:
            prec1 = accuracy(output.data[:, 532:], target3, topk=(1, ))[0]#only need top1
        elif weights[1]>0:
            prec1 = accuracy(output.data[:, 49:532], target2, topk=(1, ))[0]
        else:
            prec1 = accuracy(output.data[:, :49], target1, topk=(1, ))[0]
        top1.update(prec1[0], img.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg

In [30]:
if __name__ == '__main__':
    best_prec1 = 60
    criterion = nn.CrossEntropyLoss().cuda()
    lr = 1e-2
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr = lr, momentum=0.9, 
                          weight_decay=0)
    resume = None
    start_epoch = 0
    epochs = 2
    arch = 'resnet101_levelID'

    for epoch in range(start_epoch, epochs):
        adjust_learning_rate(lr=lr, optimizer=optimizer, epoch=epoch, denominator=2)

        # train for one epoch
        for weights in [[1, 1, 1]]:
            loss_log, acc_log = train(train_loader=loader_train, model=model, criterion=criterion,
                                      weights = weights, optimizer=optimizer, epoch=epoch)

        # evaluate on validation set
            prec1 = validate(val_loader=loader_val, model=model, weights=weights)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': arch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
            'optimizer' : optimizer.state_dict(),
        }, is_best)

        #plot loss and acc
#         fig = plt.figure(figsize = (6,3), dpi = 600)
#         loss_log = np.array(loss_log)
#         ax1 = plt.subplot(121)
#         ax1.plot(loss_log)
#         ax1.set_ylabel('Loss', weight = 'bold')
#         acc_log = np.array(acc_log)
#         ax2 = plt.subplot(111)
#         ax2.plot(acc_log)
#         ax2.set_ylabel('Train_accuracy', weight = 'bold')
#         np.savetxt(X=np.vstack((loss_log, acc_log)), fname='loss_acc_log.txt', fmt='%.3f')

Epoch: [0][0/94759]	Time 129.053 (129.053)	Data 1.500 (1.500)	Loss 6.3713 (6.3713)	Prec@1 70.312 (70.312)
Epoch: [0][50/94759]	Time 0.850 (3.360)	Data 0.000 (0.030)	Loss 5.0315 (5.8718)	Prec@1 61.719 (60.846)
Epoch: [0][100/94759]	Time 0.879 (2.117)	Data 0.000 (0.015)	Loss 4.9567 (5.6252)	Prec@1 61.719 (60.241)
Epoch: [0][150/94759]	Time 0.850 (1.701)	Data 0.000 (0.010)	Loss 5.1716 (5.4685)	Prec@1 64.062 (60.446)
Epoch: [0][200/94759]	Time 0.848 (1.489)	Data 0.000 (0.008)	Loss 5.7987 (5.3848)	Prec@1 54.688 (60.047)
Epoch: [0][250/94759]	Time 0.849 (1.363)	Data 0.000 (0.006)	Loss 5.1689 (5.2990)	Prec@1 53.906 (59.929)
Epoch: [0][300/94759]	Time 0.851 (1.278)	Data 0.000 (0.005)	Loss 4.8669 (5.2180)	Prec@1 57.031 (60.190)
Epoch: [0][350/94759]	Time 0.852 (1.217)	Data 0.000 (0.005)	Loss 4.3549 (5.1726)	Prec@1 60.938 (60.078)
Epoch: [0][400/94759]	Time 0.856 (1.171)	Data 0.000 (0.004)	Loss 4.3855 (5.1222)	Prec@1 63.281 (60.016)
Epoch: [0][450/94759]	Time 0.851 (1.136)	Data 0.000 (0.004)	Los

Epoch: [0][3950/94759]	Time 0.850 (0.884)	Data 0.000 (0.001)	Loss 4.5528 (4.5082)	Prec@1 57.812 (59.091)
Epoch: [0][4000/94759]	Time 0.850 (0.883)	Data 0.000 (0.001)	Loss 4.9867 (4.5061)	Prec@1 53.906 (59.078)
Epoch: [0][4050/94759]	Time 0.848 (0.883)	Data 0.000 (0.001)	Loss 4.0535 (4.5047)	Prec@1 55.469 (59.072)
Epoch: [0][4100/94759]	Time 0.851 (0.883)	Data 0.000 (0.001)	Loss 4.4910 (4.5025)	Prec@1 57.812 (59.060)
Epoch: [0][4150/94759]	Time 0.853 (0.882)	Data 0.000 (0.001)	Loss 3.9352 (4.4996)	Prec@1 56.250 (59.060)
Epoch: [0][4200/94759]	Time 0.852 (0.882)	Data 0.000 (0.001)	Loss 4.5045 (4.4971)	Prec@1 59.375 (59.060)
Epoch: [0][4250/94759]	Time 0.851 (0.882)	Data 0.000 (0.001)	Loss 4.5991 (4.4940)	Prec@1 58.594 (59.058)
Epoch: [0][4300/94759]	Time 0.854 (0.881)	Data 0.000 (0.001)	Loss 4.2783 (4.4918)	Prec@1 60.938 (59.055)
Epoch: [0][4350/94759]	Time 0.848 (0.881)	Data 0.000 (0.001)	Loss 4.5000 (4.4891)	Prec@1 52.344 (59.060)
Epoch: [0][4400/94759]	Time 0.853 (0.880)	Data 0.000 (0

Epoch: [0][7900/94759]	Time 0.854 (0.868)	Data 0.000 (0.000)	Loss 4.0524 (4.3904)	Prec@1 64.062 (58.809)
Epoch: [0][7950/94759]	Time 0.851 (0.868)	Data 0.000 (0.000)	Loss 3.4394 (4.3888)	Prec@1 67.969 (58.807)
Epoch: [0][8000/94759]	Time 0.851 (0.868)	Data 0.000 (0.000)	Loss 4.1491 (4.3877)	Prec@1 53.906 (58.806)
Epoch: [0][8050/94759]	Time 0.850 (0.868)	Data 0.000 (0.000)	Loss 4.0979 (4.3874)	Prec@1 61.719 (58.802)
Epoch: [0][8100/94759]	Time 0.850 (0.867)	Data 0.000 (0.000)	Loss 3.6806 (4.3856)	Prec@1 61.719 (58.805)
Epoch: [0][8150/94759]	Time 0.851 (0.867)	Data 0.000 (0.000)	Loss 4.6954 (4.3845)	Prec@1 60.156 (58.802)
Epoch: [0][8200/94759]	Time 0.851 (0.867)	Data 0.000 (0.000)	Loss 4.2897 (4.3833)	Prec@1 55.469 (58.797)
Epoch: [0][8250/94759]	Time 0.849 (0.867)	Data 0.000 (0.000)	Loss 4.8069 (4.3820)	Prec@1 58.594 (58.797)
Epoch: [0][8300/94759]	Time 0.851 (0.867)	Data 0.000 (0.000)	Loss 4.7051 (4.3810)	Prec@1 59.375 (58.796)
Epoch: [0][8350/94759]	Time 0.854 (0.867)	Data 0.000 (0

Epoch: [0][11800/94759]	Time 0.853 (0.862)	Data 0.000 (0.000)	Loss 4.4132 (4.3301)	Prec@1 54.688 (58.693)
Epoch: [0][11850/94759]	Time 0.851 (0.862)	Data 0.000 (0.000)	Loss 4.8434 (4.3297)	Prec@1 53.906 (58.692)
Epoch: [0][11900/94759]	Time 0.852 (0.862)	Data 0.000 (0.000)	Loss 3.8859 (4.3289)	Prec@1 65.625 (58.689)
Epoch: [0][11950/94759]	Time 0.850 (0.862)	Data 0.000 (0.000)	Loss 4.2908 (4.3279)	Prec@1 60.156 (58.689)
Epoch: [0][12000/94759]	Time 0.851 (0.862)	Data 0.000 (0.000)	Loss 4.1796 (4.3277)	Prec@1 59.375 (58.685)
Epoch: [0][12050/94759]	Time 0.852 (0.862)	Data 0.000 (0.000)	Loss 3.8405 (4.3272)	Prec@1 58.594 (58.685)
Epoch: [0][12100/94759]	Time 0.852 (0.862)	Data 0.000 (0.000)	Loss 3.9838 (4.3265)	Prec@1 60.938 (58.686)
Epoch: [0][12150/94759]	Time 0.849 (0.862)	Data 0.000 (0.000)	Loss 4.0304 (4.3259)	Prec@1 62.500 (58.683)
Epoch: [0][12200/94759]	Time 0.853 (0.862)	Data 0.000 (0.000)	Loss 3.7265 (4.3250)	Prec@1 57.812 (58.684)
Epoch: [0][12250/94759]	Time 0.849 (0.862)	Dat

Epoch: [0][15700/94759]	Time 0.854 (0.860)	Data 0.000 (0.000)	Loss 3.7697 (4.2894)	Prec@1 60.156 (58.647)
Epoch: [0][15750/94759]	Time 0.851 (0.860)	Data 0.000 (0.000)	Loss 3.9353 (4.2888)	Prec@1 53.906 (58.645)
Epoch: [0][15800/94759]	Time 0.852 (0.860)	Data 0.000 (0.000)	Loss 3.7626 (4.2882)	Prec@1 63.281 (58.648)
Epoch: [0][15850/94759]	Time 0.851 (0.860)	Data 0.000 (0.000)	Loss 3.5136 (4.2878)	Prec@1 64.844 (58.647)
Epoch: [0][15900/94759]	Time 0.852 (0.860)	Data 0.000 (0.000)	Loss 4.1613 (4.2873)	Prec@1 57.812 (58.647)
Epoch: [0][15950/94759]	Time 0.851 (0.860)	Data 0.000 (0.000)	Loss 5.3503 (4.2864)	Prec@1 53.906 (58.651)
Epoch: [0][16000/94759]	Time 0.850 (0.860)	Data 0.000 (0.000)	Loss 4.3402 (4.2861)	Prec@1 53.906 (58.649)
Epoch: [0][16050/94759]	Time 0.852 (0.860)	Data 0.000 (0.000)	Loss 3.8032 (4.2853)	Prec@1 65.625 (58.649)
Epoch: [0][16100/94759]	Time 0.849 (0.860)	Data 0.000 (0.000)	Loss 4.4786 (4.2843)	Prec@1 57.031 (58.652)
Epoch: [0][16150/94759]	Time 0.853 (0.860)	Dat

...

Epoch: [1][78200/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.2192 (3.5794)	Prec@1 65.625 (61.147)
Epoch: [1][78250/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.4335 (3.5793)	Prec@1 60.156 (61.148)
Epoch: [1][78300/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 4.1600 (3.5793)	Prec@1 64.062 (61.149)
Epoch: [1][78350/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.1026 (3.5792)	Prec@1 62.500 (61.149)
Epoch: [1][78400/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 4.0277 (3.5792)	Prec@1 58.594 (61.149)
Epoch: [1][78450/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 4.0965 (3.5792)	Prec@1 58.594 (61.150)
Epoch: [1][78500/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.3481 (3.5792)	Prec@1 64.844 (61.150)
Epoch: [1][78550/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.8357 (3.5792)	Prec@1 56.250 (61.150)
Epoch: [1][78600/94759]	Time 0.849 (0.852)	Data 0.000 (0.000)	Loss 4.5856 (3.5792)	Prec@1 57.031 (61.150)
Epoch: [1][78650/94759]	Time 0.852 (0.852)	Dat

Epoch: [1][82100/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.3697 (3.5767)	Prec@1 54.688 (61.162)
Epoch: [1][82150/94759]	Time 0.852 (0.852)	Data 0.000 (0.000)	Loss 3.3202 (3.5766)	Prec@1 60.938 (61.162)
Epoch: [1][82200/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 4.3607 (3.5766)	Prec@1 51.562 (61.163)
Epoch: [1][82250/94759]	Time 0.846 (0.852)	Data 0.000 (0.000)	Loss 3.2552 (3.5766)	Prec@1 67.969 (61.163)
Epoch: [1][82300/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.6834 (3.5765)	Prec@1 60.156 (61.163)
Epoch: [1][82350/94759]	Time 0.852 (0.852)	Data 0.000 (0.000)	Loss 3.1534 (3.5766)	Prec@1 60.938 (61.164)
Epoch: [1][82400/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 3.2340 (3.5765)	Prec@1 66.406 (61.164)
Epoch: [1][82450/94759]	Time 0.854 (0.852)	Data 0.000 (0.000)	Loss 4.0071 (3.5765)	Prec@1 54.688 (61.164)
Epoch: [1][82500/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 3.9735 (3.5765)	Prec@1 57.812 (61.164)
Epoch: [1][82550/94759]	Time 0.851 (0.852)	Dat

Epoch: [1][86000/94759]	Time 0.852 (0.852)	Data 0.000 (0.000)	Loss 3.5436 (3.5743)	Prec@1 61.719 (61.181)
Epoch: [1][86050/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 3.0904 (3.5744)	Prec@1 67.969 (61.180)
Epoch: [1][86100/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 4.1122 (3.5744)	Prec@1 53.906 (61.181)
Epoch: [1][86150/94759]	Time 0.856 (0.852)	Data 0.000 (0.000)	Loss 3.5160 (3.5744)	Prec@1 64.062 (61.181)
Epoch: [1][86200/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 4.4788 (3.5743)	Prec@1 47.656 (61.181)
Epoch: [1][86250/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.1164 (3.5743)	Prec@1 65.625 (61.181)
Epoch: [1][86300/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.7004 (3.5743)	Prec@1 56.250 (61.181)
Epoch: [1][86350/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.2051 (3.5742)	Prec@1 63.281 (61.182)
Epoch: [1][86400/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 3.1135 (3.5742)	Prec@1 63.281 (61.182)
Epoch: [1][86450/94759]	Time 0.852 (0.852)	Dat

Epoch: [1][89900/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.4838 (3.5716)	Prec@1 59.375 (61.199)
Epoch: [1][89950/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.8419 (3.5716)	Prec@1 62.500 (61.199)
Epoch: [1][90000/94759]	Time 0.855 (0.852)	Data 0.000 (0.000)	Loss 3.8490 (3.5715)	Prec@1 58.594 (61.199)
Epoch: [1][90050/94759]	Time 0.876 (0.852)	Data 0.000 (0.000)	Loss 4.1607 (3.5715)	Prec@1 58.594 (61.200)
Epoch: [1][90100/94759]	Time 0.853 (0.852)	Data 0.000 (0.000)	Loss 4.2426 (3.5715)	Prec@1 67.969 (61.200)
Epoch: [1][90150/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.3340 (3.5714)	Prec@1 60.156 (61.201)
Epoch: [1][90200/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.7038 (3.5714)	Prec@1 57.031 (61.201)
Epoch: [1][90250/94759]	Time 0.847 (0.852)	Data 0.000 (0.000)	Loss 3.2417 (3.5713)	Prec@1 66.406 (61.202)
Epoch: [1][90300/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.4532 (3.5712)	Prec@1 63.281 (61.203)
Epoch: [1][90350/94759]	Time 0.854 (0.852)	Dat

Epoch: [1][93800/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.1417 (3.5685)	Prec@1 68.750 (61.218)
Epoch: [1][93850/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.7995 (3.5685)	Prec@1 60.156 (61.218)
Epoch: [1][93900/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 2.5231 (3.5684)	Prec@1 78.125 (61.219)
Epoch: [1][93950/94759]	Time 0.852 (0.852)	Data 0.000 (0.000)	Loss 4.5099 (3.5684)	Prec@1 57.031 (61.219)
Epoch: [1][94000/94759]	Time 0.850 (0.852)	Data 0.000 (0.000)	Loss 3.0111 (3.5684)	Prec@1 64.062 (61.219)
Epoch: [1][94050/94759]	Time 0.855 (0.852)	Data 0.000 (0.000)	Loss 3.2635 (3.5684)	Prec@1 63.281 (61.219)
Epoch: [1][94100/94759]	Time 0.851 (0.852)	Data 0.000 (0.000)	Loss 3.2616 (3.5683)	Prec@1 63.281 (61.220)
Epoch: [1][94150/94759]	Time 0.852 (0.852)	Data 0.000 (0.000)	Loss 3.4413 (3.5682)	Prec@1 62.500 (61.220)
Epoch: [1][94200/94759]	Time 0.852 (0.852)	Data 0.000 (0.000)	Loss 4.5787 (3.5682)	Prec@1 57.812 (61.220)
Epoch: [1][94250/94759]	Time 0.855 (0.852)	Dat